In [742]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd
import pprint

# Get an overview of the data

In [743]:
# Load dataframe
df = pd.read_csv('netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [744]:
df.shape

(8807, 12)

In [745]:
df.columns.tolist()

['show_id',
 'type',
 'title',
 'director',
 'cast',
 'country',
 'date_added',
 'release_year',
 'rating',
 'duration',
 'listed_in',
 'description']

# Check columns with nulls

In [746]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [747]:
# From above info: 'director', 'country', 'cast' have most nulls consecutively
# Now confirm the column with most nulls
most_nulls_column = df.isnull().sum().idxmax()
most_nulls_column

'director'

### Dealing with null values in 'director' column

In [748]:
# Since 'director' column has most nulls, view what's the null designation
# In this case it's 'NaN'
df['director'].unique()

array(['Kirsten Johnson', nan, 'Julien Leclercq', ..., 'Majid Al Ansari',
       'Peter Hewitt', 'Mozez Singh'], dtype=object)

In [749]:
# Get the null values count
df['director'].isnull().sum()

2634

In [750]:
# Find how missing values are represented in the column: 
def missing_val_representation(column):
    missing_val = set()
    for val in column:
        if pd.isna(val) or val == "NaN" or val == "NA":
            missing_val.add(val)
    return missing_val

missing_val_representation(df['director'])

{nan}

In [751]:
# Replace null values in director column with 'Unavailable'
df['director'] = df['director'].apply(lambda x: 'Unavailable' if pd.isna(x) == True else x)

In [752]:
# Check how many null values in 'director' column
df['director'].isnull().sum()  # The count is now 0

0

In [753]:
# Check the info summary once more
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      8807 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


### Dealing with null values in 'country' column

In [754]:
# Confirm count of non-null values is equal to one above 
null_value_count_country_col = df['country'].isnull().sum()
non_null_value_count_country_col = len(df) - null_value_count_country_col

print(f"Non-null value count in 'country': {non_null_value_count_country_col}")
print(f"Null value count in 'country': {null_value_count_country_col}")

Non-null value count in 'country': 7976
Null value count in 'country': 831


In [755]:
# Replace null values in ‘country’ column with ‘Not Available’
df['country'] = df['country'].apply(lambda x: 'Unavailable' if pd.isna(x) == True else x)

In [756]:
# Confirm all null values in 'country' have been replaced
null_value_count_country_col = df['country'].isnull().sum()
print(f"Null value count 'country' col after removal: {null_value_count_country_col}")

Null value count 'country' col after removal: 0


In [757]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      8807 non-null   object
 4   cast          7982 non-null   object
 5   country       8807 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


### Dealing with null values in ‘cast’ column

In [758]:
# Confirm count of non values in 'cast' column (Hence all other values are null)
null_vals_count_cast_col = df['cast'].isnull().sum()
non_null_vals_in_cast_col = len(df) - null_vals_count_cast_col

print(f"Non-null value count in 'country': {non_null_vals_in_cast_col}")
print(f"Null value count in 'country': {null_vals_count_cast_col}")

Non-null value count in 'country': 7982
Null value count in 'country': 825


In [759]:
# Replacing all null values in 'cast' column with "Unavailable"
df['cast'] = df['cast'].apply(lambda x: "Unavailable" if pd.isna(x) == True else x)

In [760]:
# Confirm all null values in 'cast' have been removed
df['cast'].isnull().sum()

0

In [761]:
# Confirm summary info once more
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      8807 non-null   object
 4   cast          8807 non-null   object
 5   country       8807 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


### Cleaning date_added column

In [762]:
# Checking the number of null and non-null values
# Confirm count of non values in 'cast' column (Hence all other values are null)
null_vals_date_added_col = df['date_added'].isnull().sum()
non_null_vals_date_added_col = len(df) - null_vals_date_added_col

print(f"Non-null value count in 'country': {non_null_vals_date_added_col}")
print(f"Null value count in 'country': {null_vals_date_added_col}")

Non-null value count in 'country': 8797
Null value count in 'country': 10


In [763]:
# Returning rows with null values
rows_with_null_date_added_col = df[df['date_added'].isnull()]
rows_with_null_date_added_col

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
6066,s6067,TV Show,A Young Doctor's Notebook and Other Stories,Unavailable,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,NaN,2013,TV-MA,2 Seasons,"British TV Shows, TV Comedies, TV Dramas","Set during the Russian Revolution, this comic ..."
6174,s6175,TV Show,Anthony Bourdain: Parts Unknown,Unavailable,Anthony Bourdain,United States,NaN,2018,TV-PG,5 Seasons,Docuseries,This CNN original series has chef Anthony Bour...
6795,s6796,TV Show,Frasier,Unavailable,"Kelsey Grammer, Jane Leeves, David Hyde Pierce...",United States,NaN,2003,TV-PG,11 Seasons,"Classic & Cult TV, TV Comedies",Frasier Crane is a snooty but lovable Seattle ...
6806,s6807,TV Show,Friends,Unavailable,"Jennifer Aniston, Courteney Cox, Lisa Kudrow, ...",United States,NaN,2003,TV-14,10 Seasons,"Classic & Cult TV, TV Comedies",This hit sitcom follows the merry misadventure...
6901,s6902,TV Show,Gunslinger Girl,Unavailable,"Yuuka Nanri, Kanako Mitsuhashi, Eri Sendai, Am...",Japan,NaN,2008,TV-14,2 Seasons,"Anime Series, Crime TV Shows","On the surface, the Social Welfare Agency appe..."
7196,s7197,TV Show,Kikoriki,Unavailable,Igor Dmitriev,Unavailable,NaN,2010,TV-Y,2 Seasons,Kids' TV,A wacky rabbit and his gang of animal pals hav...
7254,s7255,TV Show,La Familia P. Luche,Unavailable,"Eugenio Derbez, Consuelo Duval, Luis Manuel Áv...",United States,NaN,2012,TV-14,3 Seasons,"International TV Shows, Spanish-Language TV Sh...","This irreverent sitcom featues Ludovico, Feder..."
7406,s7407,TV Show,Maron,Unavailable,"Marc Maron, Judd Hirsch, Josh Brener, Nora Zeh...",United States,NaN,2016,TV-MA,4 Seasons,TV Comedies,"Marc Maron stars as Marc Maron, who interviews..."
7847,s7848,TV Show,Red vs. Blue,Unavailable,"Burnie Burns, Jason Saldaña, Gustavo Sorola, G...",United States,NaN,2015,NR,13 Seasons,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...","This parody of first-person shooter games, mil..."
8182,s8183,TV Show,The Adventures of Figaro Pho,Unavailable,"Luke Jurevicius, Craig Behenna, Charlotte Haml...",Australia,NaN,2015,TV-Y7,2 Seasons,"Kids' TV, TV Comedies","Imagine your worst fears, then multiply them: ..."


In [764]:
# Returning null values rows together with adjacent rows to decide on the filling date
# Whether to ffill or bfill

null_indexes = df[df['date_added'].isnull()].index

rows_to_include = []

for index in null_indexes:
    if index > 0:
        rows_to_include.append(index - 1)
    rows_to_include.append(index)
    if index < len(df) - 1:
        rows_to_include.append(index + 1)

# iloc is used for integer-based indexing, selects rows at those indexes
null_date_added_rows_plus_adjacent_ones = df.iloc[rows_to_include]
null_date_added_rows_plus_adjacent_ones 

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
6065,s6066,Movie,A Wrinkle in Time,Ava DuVernay,"Storm Reid, Oprah Winfrey, Reese Witherspoon, ...",United States,"September 25, 2018",2018,PG,110 min,Children & Family Movies,"Years after their father disappears, Meg and h..."
6066,s6067,TV Show,A Young Doctor's Notebook and Other Stories,Unavailable,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,NaN,2013,TV-MA,2 Seasons,"British TV Shows, TV Comedies, TV Dramas","Set during the Russian Revolution, this comic ..."
6067,s6068,TV Show,A.D. Kingdom and Empire,Unavailable,"Juan Pablo Di Pace, Adam Levy, Chipo Chung, Ba...",United States,"December 15, 2017",2015,TV-14,1 Season,TV Dramas,"In the wake of Jesus Christ's crucifixion, his..."
6173,s6174,Movie,Antariksha Ke Rakhwale,Anirban Majumder,"Sonal Kaushal, Rupa Bhimani, Julie Tejwani, Sa...",Unavailable,"June 18, 2019",2018,TV-Y7,65 min,Children & Family Movies,Space villains have abducted all the superhero...
6174,s6175,TV Show,Anthony Bourdain: Parts Unknown,Unavailable,Anthony Bourdain,United States,NaN,2018,TV-PG,5 Seasons,Docuseries,This CNN original series has chef Anthony Bour...
6175,s6176,Movie,Antidote,Ken Barbet,"Randy Couture, Chuck Zito, Wil Traval, Celeste...",United States,"April 14, 2019",2018,TV-MA,90 min,"Action & Adventure, Horror Movies",A tough-as-nails treasure hunter protects a hu...
6794,s6795,Movie,Frances Ha,Noah Baumbach,"Greta Gerwig, Mickey Sumner, Charlotte d'Amboi...","United States, Brazil","October 1, 2017",2012,R,86 min,"Comedies, Dramas, Independent Movies",Determined to make it as a modern dancer in Ne...
6795,s6796,TV Show,Frasier,Unavailable,"Kelsey Grammer, Jane Leeves, David Hyde Pierce...",United States,NaN,2003,TV-PG,11 Seasons,"Classic & Cult TV, TV Comedies",Frasier Crane is a snooty but lovable Seattle ...
6796,s6797,Movie,Freak Show,Trudie Styler,"Alex Lawther, Abigail Breslin, AnnaSophia Robb...",United States,"August 13, 2020",2018,TV-MA,91 min,"Comedies, Dramas, Independent Movies","Forced to attend a new high school, a glamorou..."
6805,s6806,Movie,Friend Request,Simon Verhoeven,"Alycia Debnam-Carey, William Moseley, Connor P...",Germany,"April 18, 2018",2016,R,92 min,"Horror Movies, International Movies",A popular college student's love of social med...


In [765]:
# Getting the indexes of the null values
date_added_null_values_indexes = df[df['date_added'].isnull()].index
date_added_null_values_indexes

Index([6066, 6174, 6795, 6806, 6901, 7196, 7254, 7406, 7847, 8182], dtype='int64')

In [766]:
# Check date format
df['date_added'].head()

0    September 25, 2021
1    September 24, 2021
2    September 24, 2021
3    September 24, 2021
4    September 24, 2021
Name: date_added, dtype: object

In [767]:
# Check if it's type date
type(df['date_added'][0])

str

In [768]:
# Since it's a string. Convert values in the column to datetime objects
# df['date_added'] = pd.to_datetime(df['date_added'], format='%B %d, %Y', errors="coerce")
df['date_added'] = pd.to_datetime(df['date_added'], format='mixed', dayfirst=True)

# Check if type has changed
# type(df['date_added'][0])

In [769]:
df['date_added'].unique()

<DatetimeArray>
['2021-09-25 00:00:00', '2021-09-24 00:00:00', '2021-09-23 00:00:00',
 '2021-09-22 00:00:00', '2021-09-21 00:00:00', '2021-09-20 00:00:00',
 '2021-09-19 00:00:00', '2021-09-17 00:00:00', '2021-09-16 00:00:00',
 '2021-09-15 00:00:00',
 ...
 '2018-09-27 00:00:00', '2017-03-23 00:00:00', '2016-12-25 00:00:00',
 '2016-11-30 00:00:00', '2017-10-23 00:00:00', '2017-11-04 00:00:00',
 '2015-08-05 00:00:00', '2018-12-06 00:00:00', '2016-03-09 00:00:00',
 '2020-01-11 00:00:00']
Length: 1715, dtype: datetime64[ns]

In [770]:
# Confirming that only the original null values indexes still exist
date_added_null_values_indexes = df[df['date_added'].isnull()].index
date_added_null_values_indexes

Index([6066, 6174, 6795, 6806, 6901, 7196, 7254, 7406, 7847, 8182], dtype='int64')

In [771]:
# Getting the count of null values
null_vals_date_added_col = df['date_added'].isnull().sum()
null_vals_date_added_col

10

In [772]:
# Checking the type of converted time column values
type(df['date_added'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [773]:
# Interpolate the null date values
df['date_added'] = df['date_added'].interpolate(method='linear')

In [774]:
# Getting the count of null values
null_vals_date_added_col = df['date_added'].isnull().sum()
null_vals_date_added_col 

0

In [775]:
# iloc is used for integer-based indexing, selects rows at those indexes
null_date_added_rows_plus_adjacent_ones = df.iloc[rows_to_include]
null_date_added_rows_plus_adjacent_ones 

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
6065,s6066,Movie,A Wrinkle in Time,Ava DuVernay,"Storm Reid, Oprah Winfrey, Reese Witherspoon, ...",United States,2018-09-25 00:00:00,2018,PG,110 min,Children & Family Movies,"Years after their father disappears, Meg and h..."
6066,s6067,TV Show,A Young Doctor's Notebook and Other Stories,Unavailable,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,2018-05-06 00:00:00,2013,TV-MA,2 Seasons,"British TV Shows, TV Comedies, TV Dramas","Set during the Russian Revolution, this comic ..."
6067,s6068,TV Show,A.D. Kingdom and Empire,Unavailable,"Juan Pablo Di Pace, Adam Levy, Chipo Chung, Ba...",United States,2017-12-15 00:00:00,2015,TV-14,1 Season,TV Dramas,"In the wake of Jesus Christ's crucifixion, his..."
6173,s6174,Movie,Antariksha Ke Rakhwale,Anirban Majumder,"Sonal Kaushal, Rupa Bhimani, Julie Tejwani, Sa...",Unavailable,2019-06-18 00:00:00,2018,TV-Y7,65 min,Children & Family Movies,Space villains have abducted all the superhero...
6174,s6175,TV Show,Anthony Bourdain: Parts Unknown,Unavailable,Anthony Bourdain,United States,2019-05-16 12:00:00,2018,TV-PG,5 Seasons,Docuseries,This CNN original series has chef Anthony Bour...
6175,s6176,Movie,Antidote,Ken Barbet,"Randy Couture, Chuck Zito, Wil Traval, Celeste...",United States,2019-04-14 00:00:00,2018,TV-MA,90 min,"Action & Adventure, Horror Movies",A tough-as-nails treasure hunter protects a hu...
6794,s6795,Movie,Frances Ha,Noah Baumbach,"Greta Gerwig, Mickey Sumner, Charlotte d'Amboi...","United States, Brazil",2017-10-01 00:00:00,2012,R,86 min,"Comedies, Dramas, Independent Movies",Determined to make it as a modern dancer in Ne...
6795,s6796,TV Show,Frasier,Unavailable,"Kelsey Grammer, Jane Leeves, David Hyde Pierce...",United States,2019-03-08 12:00:00,2003,TV-PG,11 Seasons,"Classic & Cult TV, TV Comedies",Frasier Crane is a snooty but lovable Seattle ...
6796,s6797,Movie,Freak Show,Trudie Styler,"Alex Lawther, Abigail Breslin, AnnaSophia Robb...",United States,2020-08-13 00:00:00,2018,TV-MA,91 min,"Comedies, Dramas, Independent Movies","Forced to attend a new high school, a glamorou..."
6805,s6806,Movie,Friend Request,Simon Verhoeven,"Alycia Debnam-Carey, William Moseley, Connor P...",Germany,2018-04-18 00:00:00,2016,R,92 min,"Horror Movies, International Movies",A popular college student's love of social med...


In [776]:
# Converting date_added col from Timestamp (date & time) to date only
# after interpolating the null values
df['date_added'] = df['date_added'].dt.date

# Now checking the type of the converted column
type(df['date_added'][0])

datetime.date

In [777]:
# Viewing the converted types
df['date_added'].head()

0    2021-09-25
1    2021-09-24
2    2021-09-24
3    2021-09-24
4    2021-09-24
Name: date_added, dtype: object

In [778]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      8807 non-null   object
 4   cast          8807 non-null   object
 5   country       8807 non-null   object
 6   date_added    8807 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


## Dropping the null rows

In [779]:
df.dropna(inplace=True)

In [780]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8800 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8800 non-null   object
 1   type          8800 non-null   object
 2   title         8800 non-null   object
 3   director      8800 non-null   object
 4   cast          8800 non-null   object
 5   country       8800 non-null   object
 6   date_added    8800 non-null   object
 7   release_year  8800 non-null   int64 
 8   rating        8800 non-null   object
 9   duration      8800 non-null   object
 10  listed_in     8800 non-null   object
 11  description   8800 non-null   object
dtypes: int64(1), object(11)
memory usage: 893.8+ KB


In [781]:
# Convert movies duration to minutes
df.duration.head()

0       90 min
1    2 Seasons
2     1 Season
3     1 Season
4    2 Seasons
Name: duration, dtype: object

## Drop the show_id column 

In [782]:
df = df.drop(['show_id'], axis=1)

## Cleaning the 'country' column

In [783]:
# Check first few columns first
df['country'].head(10)

0                                        United States
1                                         South Africa
2                                          Unavailable
3                                          Unavailable
4                                                India
5                                          Unavailable
6                                          Unavailable
7    United States, Ghana, Burkina Faso, United Kin...
8                                       United Kingdom
9                                        United States
Name: country, dtype: object

In [784]:
df.head(10)

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unavailable,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,TV Show,Blood & Water,Unavailable,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Unavailable,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,TV Show,Jailbirds New Orleans,Unavailable,Unavailable,Unavailable,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,TV Show,Kota Factory,Unavailable,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",Unavailable,2021-09-24,2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
6,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",Unavailable,2021-09-24,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,2021-09-24,2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,2021-09-24,2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...


In [ ]:
df[df['title'] == "Sankofa"]

# DATA VISUALIZATION SECTION

## 1) Understanding what content is available in different countries.

### Plotting country counts

In [ ]:
# Show the counts of different countries
df['country'].nunique()

In [ ]:
df['country'].unique()

In [ ]:
# # Create a dataframe to represent the actual counts of countries since some rows.... 
# # ... in the country column have several countries in them
# one_country_per_row_df = (df['country'].str.split(', ', expand=True)
#                           .stack().reset_index(level=1,drop=True))
# 
# one_country_per_row_df = pd.DataFrame(one_country_per_row_df, 
#                                       columns=['country']).reset_index(drop=True)
# one_country_per_row_df['country'].unique()

In [ ]:
# # Find highest count among the countries
# countries = one_country_per_row_df['country'].value_counts()
# country_idx = countries.index
# country_vals_max = countries.values.max()
# # country_vals_max
# countries

In [ ]:
# # For proper smaller plot, cluster countries with count less than 100 together
# countries_counts_less_than_100 = countries[countries < 100]
# print(f"No. of countries with counts less than 100: {len(countries_counts_less_than_100)}")

In [ ]:
# # Rename countries with counts less than 10 as 'Others'
# one_country_per_row_df['country'] = one_country_per_row_df['country'].apply(lambda x: "Others" if x in countries_counts_less_than_100 else x)
# 
# one_country_per_row_df['country'].value_counts()

In [ ]:
# # Plotting the counts of different countries
# # Chart size definition:
# plt.figure(figsize=(30,10))
# 
# # Extract counts (indexes & values)
# countries = one_country_per_row_df['country'].value_counts()
# country_idx = countries.index
# country_vals = countries.values
# 
# # Defining chart labels
# plt.xlabel("Country")
# plt.xticks(rotation=90)
# plt.ylabel("Counts")
# plt.title("Countries Counts in the Netflix Shows Dataset")
# 
# # Plotting
# plt.bar(country_idx, country_vals, label='country')
# plt.legend()
# 
# # plt.savefig('country_counts.png',bbox_inches="tight",pad_inches=2,transparent=False)

In [ ]:
df.columns

In [ ]:
df.head(10)

In [ ]:
# # Check popularity of TV shows vs Movies in the various countries
# my_short_df = df.iloc[5:10]
# type(df.iloc[0:11])
# 
# my_short_df

In [ ]:
# my_short_df.to_csv("movies.csv")